- Dataframe creation of the table event_games and the table players_current_month in the silver DB, using Spark

In [0]:
players_df = spark.table('silver.players_current_month')
games_df = spark.table('silver.event_games')

- Making a join of the games and players tables, using the fide id as the connector, and adding the country column for black and white players at the games table

In [0]:
from pyspark.sql import functions as F

# Alias the DataFrames for clarity
games_df_aliased = games_df.alias("games")
players_df_white = players_df.alias("players_white")
players_df_black = players_df.alias("players_black")

# Join partidas_df with players_df (aliased as players_white) on white_fide_id to get white_country
joined_df = games_df_aliased.join(
    players_df_white,
    F.col("games.white_fide_id") == F.col("players_white.fideid"),
    "left_outer"
).select(
    F.col("games.*"),
    F.col("players_white.country").alias("white_country")
)

# Join the result with players_df (aliased as players_black) on black_fide_id to get black_country
joined_df = joined_df.join(
    players_df_black,
    F.col("games.black_fide_id") == F.col("players_black.fideid"),
    "left_outer"
).select(
    F.col("games.*"),
    F.col("white_country"),
    F.col("players_black.country").alias("black_country")
)

- Ordering the columns

In [0]:
# Adjusting the order of columns in joined_df, placing white_country right after white and black_country right after black
ordered_columns = [
    col for col in joined_df.columns if col not in ["white_country", "black_country"]
]

# Assuming the columns to place after are named "white" and "black" respectively
white_index = ordered_columns.index("white") + 1
black_index = ordered_columns.index("black") + 1

# Inserting the country columns in the desired positions
ordered_columns.insert(white_index, "white_country")
ordered_columns.insert(black_index + 1, "black_country")  # +1 to account for the insertion of white_country

# Selecting columns in the new order
joined_df = joined_df.select(*ordered_columns)

In [0]:
joined_df.write.mode('overwrite').saveAsTable('silver.games_with_country')

In [0]:
ind_df = joined_df.filter(
    F.col("white_country") == "IND"
)

display(ind_df)

In [0]:
ind_df.write.mode('overwrite').saveAsTable('silver.games_IND')

Filtering for white_country 'IND' and counting each opening, then ordering by count descending

In [0]:
openings_count_df = joined_df.filter(
    F.col("white_country") == "IND"
).groupBy("opening").agg(
    F.count("*").alias("openings_count")
).orderBy(
    F.col("openings_count").desc()
)

# Display the result
display(openings_count_df)

opening,openings_count
Sicilian,1142
French,340
Caro-Kann,321
Ruy Lopez,320
Queen's pawn game,314
Reti opening,303
King's Indian,265
Reti,263
QGD,241
English opening,231


- Filtering for white_country 'IND' and result_of_match 'White wins', counting each opening, and displaying each unique result_of_match

In [0]:
from pyspark.sql import functions as F

openings_results_df = joined_df.filter(
    (F.col("white_country") == "IND") & 
    (F.col("result_of_match") == "White wins")
).groupBy("opening", "result_of_match").agg(
    F.count("*").alias("openings_count")
).orderBy(
    F.col("openings_count").desc(), F.col("opening")
)

# Display the result
display(openings_results_df)

opening,result_of_match,openings_count
Sicilian,White wins,497
Reti opening,White wins,176
French,White wins,151
Ruy Lopez,White wins,142
King's Indian,White wins,141
Caro-Kann,White wins,134
Reti,White wins,132
Queen's pawn game,White wins,127
QGD,White wins,120
English opening,White wins,114


- Calculating the win percentage for each opening of the indian players, where the white_wins_count is over 100, for a more accurate statistic 

In [0]:
# Calculate total openings count for normalization
total_openings_df = joined_df.filter(
    F.col("white_country") == "IND"
).groupBy("opening").agg(
    F.count("*").alias("total_openings_count")
)

# Calculate white wins count and percentage
white_wins_percentage_df_ind = joined_df.filter(
    (F.col("white_country") == "IND") & 
    (F.col("result_of_match") == "White wins")
).groupBy("opening").agg(
    F.count("*").alias("victory_count_ind")
).join(
    total_openings_df, ["opening"], "inner"
).withColumn(
    "percentage_ind", 
    F.round((F.col("victory_count_ind") / F.col("total_openings_count") * 100), 1)
).filter(
    F.col("victory_count_ind") > 100
).select(
    "opening", "victory_count_ind", "percentage_ind"
).orderBy(
    F.col("percentage_ind").desc()
)

# Display the result
display(white_wins_percentage_df_ind)

opening,victory_count_ind,percentage_ind
Reti opening,176,58.1
English,107,53.8
King's Indian,141,53.2
Reti,132,50.2
QGD,120,49.8
English opening,114,49.4
Ruy Lopez,142,44.4
French,151,44.4
Sicilian,497,43.5
Caro-Kann,134,41.7


In [0]:
white_wins_percentage_df_ind.write.mode('overwrite').saveAsTable('gold.IND_most_ef_openings')

- Calculating the win percentage for each opening of the world players, where the white_wins_count is over 100, for a more accurate statistic

In [0]:
# Calculate total openings count for normalization
total_openings_df = joined_df.groupBy("opening").agg(
    F.count("*").alias("total_openings_count")
)

# Calculate white wins count and percentage
white_wins_percentage_df_world = joined_df.filter(
    F.col("result_of_match") == "White wins"
).groupBy("opening").agg(
    F.count("*").alias("white_wins_count")
).join(
    total_openings_df, ["opening"], "inner"
).withColumn(
    "percentage", 
    F.round((F.col("white_wins_count") / F.col("total_openings_count") * 100), 1)
).filter(
    F.col("white_wins_count") > 100
).select(
    "opening", "white_wins_count", "percentage"
).orderBy(
    F.col("percentage").desc()
)

# Display the result
display(white_wins_percentage_df_world)

opening,white_wins_count,percentage
Anti-Borg (Desprez) opening,109,68.6
King's pawn opening,478,63.2
Anderssen's opening,220,59.3
"Neo-Gruenfeld, 6.O-O c6",119,54.1
"Dunst (Sleipner, Heinrichsen) opening",141,53.8
"Queen's pawn, Mason variation",409,53.4
Four knights game,150,52.6
Vienna,132,52.4
Polish (Sokolsky) opening,135,51.7
Benko gambit half accepted,128,51.0


In [0]:
# Rename columns in white_wins_percentage_df_world
white_wins_percentage_df_world = white_wins_percentage_df_world.withColumnRenamed(
    "white_wins_count", "victory_count_world"
).withColumnRenamed(
    "percentage", "percentage_world"
)

# Display the result
display(white_wins_percentage_df_world)

opening,victory_count_world,percentage_world
Anti-Borg (Desprez) opening,109,68.6
King's pawn opening,478,63.2
Anderssen's opening,220,59.3
"Neo-Gruenfeld, 6.O-O c6",119,54.1
"Dunst (Sleipner, Heinrichsen) opening",141,53.8
"Queen's pawn, Mason variation",409,53.4
Four knights game,150,52.6
Vienna,132,52.4
Polish (Sokolsky) opening,135,51.7
Benko gambit half accepted,128,51.0


- Joining dataframes for openings compare

In [0]:
# Join the dataframes on the 'opening' column
openings_comp_ind_world = white_wins_percentage_df_world.join(
    white_wins_percentage_df_ind, 
    on="opening", 
    how="inner"
).select(
    "opening", 
    "victory_count_world", 
    "victory_count_ind", 
    "percentage_world", 
    "percentage_ind"
).orderBy(
    F.col("percentage_ind").desc()
)

# Display the result
display(openings_comp_ind_world)

opening,victory_count_world,victory_count_ind,percentage_world,percentage_ind
Reti opening,2038,176,47.3,58.1
English,2124,107,45.6,53.8
King's Indian,2679,141,47.0,53.2
Reti,1658,132,45.7,50.2
QGD,3126,120,45.3,49.8
English opening,1707,114,45.2,49.4
Ruy Lopez,1946,142,40.2,44.4
French,3423,151,44.5,44.4
Sicilian,10063,497,43.7,43.5
Caro-Kann,2936,134,44.2,41.7


In [0]:
openings_comp_ind_world.write.mode('overwrite').saveAsTable('gold.IND_world_comp_openings')